We'll use machine learning to predict the sale prices of houses in Ames, Iowa using data collected from 2006 to 2010.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

In [2]:
Housing = pd.read_csv('AmesHousing.tsv', sep='\t')

In [3]:
def transform_features(Housing):
    return Housing

def select_features(Housing):
    return Housing[['Gr Liv Area', 'SalePrice']]

In [4]:
def train_and_test(data):
    train = data[:1460]
    test = data[1460:]
    num_train = train.select_dtypes(include=['float', 'int'])
    num_test = test.select_dtypes(include=['float', 'int'])
    target = 'SalePrice'
    features = num_train.columns.drop([target])

    model = linear_model.LinearRegression()
    model.fit(num_train[features], num_train[target])

    train_predictions = model.predict(num_train[features])
    test_predictions = model.predict(test[features])

    train_mse = mean_squared_error(train_predictions, train[target])
    test_mse = mean_squared_error(test_predictions, test[target])

    train_rmse = np.sqrt(train_mse)
    test_rmse = np.sqrt(test_mse)
    print(train_rmse)
    print(test_rmse)

    
transformed_df = transform_features(Housing)
filtered_df = select_features(transformed_df)
rmse = train_and_test(filtered_df)

56034.362001412796
57088.25161263909


Let's see the number of missing values and drop any columns with more than 5% missing values.

In [5]:
house_null_counts = Housing.isnull().sum()
drop_null_house = house_null_counts[(house_null_counts > len(Housing)/20) & (house_null_counts > 0)].sort_values(ascending=True)
drop_null_house

Garage Type       157
Garage Yr Blt     159
Garage Finish     159
Garage Qual       159
Garage Cond       159
Lot Frontage      490
Fireplace Qu     1422
Fence            2358
Alley            2732
Misc Feature     2824
Pool QC          2917
dtype: int64

We will fill in the missing values that are 5% or less of the missing values of the columns with the mode of the column.

In [6]:
drop_null_house = drop_null_house.fillna(drop_null_house.mode)
drop_null_house.isnull().sum()

0

In [7]:
Housing.columns

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
      

Let's look at the columns that are remaining and see what new features we can create. We can look at the difference between the remodeling year and the sale year, and the year the house was built and the year it was sold.

In [8]:
yrs_since_remodel = Housing['Yr Sold'] - Housing['Year Remod/Add']
house_age = Housing['Yr Sold'] - Housing['Year Built']

#check for negative values for years since remodel and the house age
print(yrs_since_remodel[yrs_since_remodel < 0])
print(house_age[house_age < 0])

1702   -1
2180   -2
2181   -1
dtype: int64
2180   -1
dtype: int64


We will delete the negative values for the years since the remodel and the age of the house.

In [9]:
#1. Create new columns
Housing['Years Since Remodel'] = yrs_since_remodel
Housing['House Age'] = house_age

#2. Drop negative values from the dataframe
Housing = Housing.drop([1702, 2180, 2181], axis=0)

#3. Delete original year columns from dataframe since no longer needed
Housing = Housing.drop(['Year Remod/Add', 'Year Built'], axis=1)

Now we will remove features we do not want in our model such as those with leak information about the house sale (i.e. 'Yr Sold').

In [10]:
drop_cols = ['Mo Sold', 'Yr Sold', 'Sale Type', 'Sale Condition']

house = Housing[Housing.columns.drop(drop_cols)]
house.columns

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Roof Style', 'Roof Matl', 'Exterior 1st',
       'Exterior 2nd', 'Mas Vnr Type', 'Mas Vnr Area', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2',
       'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air',
       'Electrical', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Fireplace Qu',
       'Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Cars',
      

Garage Cond and Garage Qual are redundant. Misc Feature and Misc Val are not useful for machine learning. Thus we will drop Garage Cond, Misc Feature, and Misc Val.

In [11]:
house = house.drop(['Garage Cond', 'Misc Feature', 'Misc Val'], axis=1)
house.columns

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Roof Style', 'Roof Matl', 'Exterior 1st',
       'Exterior 2nd', 'Mas Vnr Type', 'Mas Vnr Area', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2',
       'Bsmt Unf SF', 'Total Bsmt SF', 'Heating', 'Heating QC', 'Central Air',
       'Electrical', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF',
       'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Kitchen Qual',
       'TotRms AbvGrd', 'Functional', 'Fireplaces', 'Fireplace Qu',
       'Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Cars',
      

In order to visually see the correlation between numerical features we will generate a heatmap matrix. we will assume a correlation above 0.3 is a strong one.

In [12]:
import seaborn as sns

correlation_matrix = house.corr()
sorted_corrs = correlation_matrix['SalePrice'].abs().sort_values(ascending=True)
sorted_corrs

BsmtFin SF 2           0.006001
Order                  0.031542
3Ssn Porch             0.032268
Bsmt Half Bath         0.035894
Low Qual Fin SF        0.037629
Pool Area              0.068438
MS SubClass            0.085128
Overall Cond           0.101540
Screen Porch           0.112280
Kitchen AbvGr          0.119760
Enclosed Porch         0.128685
Bedroom AbvGr          0.143916
Bsmt Unf SF            0.182299
PID                    0.246389
Lot Area               0.267520
2nd Flr SF             0.269601
Bsmt Full Bath         0.276486
Half Bath              0.284871
Open Porch SF          0.316262
Wood Deck SF           0.328183
Lot Frontage           0.357718
BsmtFin SF 1           0.439051
Fireplaces             0.474831
TotRms AbvGrd          0.498574
Mas Vnr Area           0.513121
Garage Yr Blt          0.527047
Years Since Remodel    0.534985
Full Bath              0.546118
House Age              0.558979
1st Flr SF             0.635185
Garage Area            0.641690
Total Bs

We will consider strong correlations as those of 0.3 and higher.

In [13]:
strong_corrs = sorted_corrs[sorted_corrs > 0.3]
corrmat = house[strong_corrs.index].corr()
sns.heatmap(corrmat)
plt.show()

/dataquest/system/env/python3/lib/python3.4/site-packages/plotly/matplotlylib/renderer.py:451: UserWarning:

Dang! That path collection is out of this world. I totally don't know what to do with it yet! Plotly can only import path collections linked to 'data' coordinates



In [14]:
strong_corrs

Open Porch SF          0.316262
Wood Deck SF           0.328183
Lot Frontage           0.357718
BsmtFin SF 1           0.439051
Fireplaces             0.474831
TotRms AbvGrd          0.498574
Mas Vnr Area           0.513121
Garage Yr Blt          0.527047
Years Since Remodel    0.534985
Full Bath              0.546118
House Age              0.558979
1st Flr SF             0.635185
Garage Area            0.641690
Total Bsmt SF          0.643782
Garage Cars            0.648427
Gr Liv Area            0.717596
Overall Qual           0.801206
SalePrice              1.000000
Name: SalePrice, dtype: float64

The following columns are categorical: BsmtFin SF 1, Garage Yr Blt, and Garage Cars. Let's compute the variance for the columns in our dataframe and see which features have low variance.

In [15]:
print(house.var().sort_values(ascending=True))

Kitchen AbvGr          4.587361e-02
Bsmt Half Bath         5.990693e-02
Half Bath              2.525001e-01
Bsmt Full Bath         2.754340e-01
Full Bath              3.052255e-01
Fireplaces             4.194511e-01
Garage Cars            5.779952e-01
Bedroom AbvGr          6.855773e-01
Overall Cond           1.236455e+00
Overall Qual           1.981361e+00
TotRms AbvGrd          2.444504e+00
Years Since Remodel    4.349031e+02
Lot Frontage           5.419487e+02
3Ssn Porch             6.327277e+02
Garage Yr Blt          6.514806e+02
House Age              9.171194e+02
Pool Area              1.268453e+03
MS SubClass            1.819383e+03
Low Qual Fin SF        2.146840e+03
Screen Porch           3.148756e+03
Enclosed Porch         4.117493e+03
Open Porch SF          4.414411e+03
Wood Deck SF           1.590622e+04
BsmtFin SF 2           2.864479e+04
Mas Vnr Area           3.152674e+04
Garage Area            4.605354e+04
1st Flr SF             1.470394e+05
2nd Flr SF             1.831

The columns with the lowest variances are Kitchen AbvGr and Bsmt Half Bath so we'll drop these columns as well. We'll also drop PID as it has a very high variance.

In [16]:
house = house.drop(['Kitchen AbvGr', 'Bsmt Half Bath', 'PID'], axis =1)

Now let's update the select_features() function. It will take in the new, modified train and test data frames that were returned from transform_features().

In [17]:
def transform_features(Housing):
    #count and drop columns with more than 5% missing values
    house_null_counts = Housing.isnull().sum()
    drop_null_house = house_null_counts[(house_null_counts > len(Housing)/20) & (house_null_counts > 0)].sort_values(ascending=True)
    
    #fill in column mode for columns with less than 5% missing values
    drop_null_house = drop_null_house.fillna(drop_null_house.mode)
    
    #create new, informative features
    yrs_since_remodel = Housing['Yr Sold'] - Housing['Year Remod/Add']
    house_age = Housing['Yr Sold'] - Housing['Year Built']
    
    #1. Create new columns for the new features
    Housing['Years Since Remodel'] = yrs_since_remodel
    Housing['House Age'] = house_age

    #2. Drop negative values from the dataframe
    Housing = Housing.drop([1702, 2180, 2181], axis=0)

    #3. Delete original year columns from dataframe since no longer needed
    Housing = Housing.drop(['Year Remod/Add', 'Year Built'], axis=1)    
    
    #remove columns that leak information aboutthe target column
    drop_cols = ['Mo Sold', 'Yr Sold', 'Sale Type', 'Sale Condition']
    Housing = Housing[Housing.columns.drop(drop_cols)]

    return Housing

def select_features(Housing):
    
    return Housing[['Gr Liv Area', 'SalePrice']]

def train_and_test(Housing):
    train = Housing[:1460]
    test = Housing[1460:]
    num_train = train.select_dtypes(include=['float', 'int'])
    num_test = test.select_dtypes(include=['float', 'int'])
    target = 'SalePrice'
    features = num_train.columns.drop([target])

    model = linear_model.LinearRegression()
    model.fit(num_train[features], num_train[target])

    predictions = model.predict(test[features])
    mse = mean_squared_error(predictions, test[target])
    rmse = np.sqrt(mse)
    return rmse

Housing = pd.read_csv('AmesHousing.tsv', sep='\t')    
transformed_df = transform_features(Housing)
filtered_df = select_features(transformed_df)
rmse = train_and_test(filtered_df)
rmse

55275.36731241307

In [20]:
def train_and_test(Housing, model, k=0):
    num_df = Housing.select_dtypes(include=['float', 'int'])
    target = 'SalePrice'
    features = num_df.columns.drop([target])
    #model = linear_model.LinearRegression()
    #model = ensemble.Rand
    if k == 0:
        train = Housing[:1460]
        test = Housing[1460:]
        model.fit(train_one[[features]], train[target])
        predictions = model.predict(test[features])
        mse = mean_squared_error(test[target], predictions)
        rmse = np.sqrt(mse)
    elif k == 1:
        shuffled_df = Housing.sample(frac=1, )
        fold_one = Housing[:1460]
        fold_two = Housing[1460:]
        model.fit(fold_one[[features]], fold_one[target])
        predictions = model.predict(fold_two[[features]])
        mse_one = mean_squared_error(fold_two[target], predictions)
        rmse_one = sqrt(mse)
    
        model.fit(fold_two[[features]], fold_two[target])
        predictions = model.predict(fold_one[[features]])
        mse_two = mean_squared_error(fold_one[target], predictions)
        rmse_two = sqrt(mse)
        rmse = np.mean(rmse_one, rmse_two)
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmses = []
        for train_index, test_index in kf.split(Housing):
            train = Housing.iloc[train_index]
            test = Housing.iloc[test_index]
            model.fit(train[features], train[target])
            predictions = model.predict(test[features])
            mse_k = mean_squared_error(test[target], predictions)
            rmse_k = np.sqrt(mse_k)
            rmses.append(rmse_k)
    rmse = np.mean(rmses)
    print(rmses)
    var_k = np.std(rmses)        
    return rmse, var_k

# Housing = pd.read_csv('AmesHousing.tsv', sep='\t')    
# transformed_df = transform_features(Housing)
# filtered_df = select_features(transformed_df)
# rmse = train_and_test(filtered_df, model, k=4)
# rmse

In [21]:
Housing = pd.read_csv('AmesHousing.tsv', sep='\t')    
transformed_df = transform_features(Housing)
filtered_df = select_features(transformed_df)
lr = LinearRegression()
rf = RandomForestRegressor()
ls = linear_model.LassoCV()
ri = linear_model.Ridge(alpha=0.1)
models = {'linear_reg': lr, 'random forest': rf, 'lasso': ls, 'ridge': ri}
model_results = {}

for name, model in models.items():
    rmse = train_and_test(filtered_df, model, k=4)
    model_results[name] = rmse
model_results

[54064.574945277534, 53936.48131965016, 58755.24514756473, 55851.5200363056]
[64962.564046149506, 61444.28824869027, 69278.05018096384, 69108.16983504481]
[53516.78077096286, 56281.838611129475, 56249.19304338287, 57030.99951796036]
[55131.62457096905, 57218.85439521964, 56248.4839438361, 54477.66290174242]


{'lasso': (55769.1564529418, 1049.497672781238),
 'linear_reg': (55769.702985858894, 1337.7893203153155),
 'random forest': (66198.26807771211, 3243.450119440879),
 'ridge': (55651.95536219951, 1945.0489280344084)}

Lasso is the best model: low average rmse and lowest variation.